# Tensorboard

https://pytorch.org/docs/stable/tensorboard.html

## Build a common network

In [1]:
# imports
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [10]:
name = 'test2'
batch_size = 128
shuffle = True
lr = 0.001

In [11]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# datasets
trainset = torchvision.datasets.FashionMNIST('../../pytorch_study_data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.FashionMNIST('../../pytorch_study_data',
    download=True,
    train=False,
    transform=transform)

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=shuffle, num_workers=2)


testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

# constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# helper function to show an image
# (used in the `plot_classes_preds` function below)
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.cpu().data.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net().to(device)

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

## Add (something)

### Graph

We can see our networks with `writer.add_graph(net, img)`. In this part, `img` is used for input of the network. `img` makes us be able to trace inputs.

In [6]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
# writer = SummaryWriter() # 기본적으로 runs에 날짜, 사용된 컴퓨터의 정보가 들어감
# writer = SummaryWriter('runs/fashion_mnist_experiment_1') # 직접 경로 조정
comment = f" name = {name} batch_size = {batch_size} lr = {lr} shuffle = {shuffle}"
writer = SummaryWriter(comment=comment)

In [7]:
img, _ = trainloader.dataset[0]
img = img.to(device)
writer.add_graph(net, img.unsqueeze(0))
writer.close()

### Scalar
### Image
### Parameters (histograms / distributions)
### Hyperparameters

In [12]:
from tqdm import tqdm

comment = f" name = {name} batch_size = {batch_size} lr = {lr} shuffle = {shuffle}"
writer = SummaryWriter(comment=comment)

running_loss = 0.0
running_loss_test = 0.0
print_freq = 10
for epoch in range(3):  # loop over the dataset multiple times
    for idx, batch in tqdm(enumerate(trainloader), total=len(trainloader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if idx % print_freq == 9:  # every print_freq mini-batches...
            # ...log the running loss
            # Write scaler to Loss - train directory
            writer.add_scalar(
                "Loss/train", 
                running_loss / print_freq, 
                epoch * len(trainloader) + idx,
            )
            writer.add_image("Images", torchvision.utils.make_grid(inputs))
            running_loss = 0.0

    total_correct = 0
    for idx, batch in tqdm(enumerate(testloader), total=len(testloader)):
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        running_loss_test += loss
        total_correct += torch.sum(torch.argmax(outputs, dim=1) == labels)

        if idx % print_freq == 9:  # every print_freq mini-batches...
            # ...log the running loss
            # Write scaler to Loss - test directory
            writer.add_scalar(
                "Loss/test",
                running_loss_test / print_freq,
                epoch * len(testloader) + idx,
            )
            running_loss = 0.0

    # Write images
    writer.add_image("Images", torchvision.utils.make_grid(inputs))
    # Another scalar
    writer.add_scalar("Accuracy", total_correct / len(testset), epoch)
    
    # We can visualize parameter distribution and histogram!
    for name, param in net.named_parameters():
        writer.add_histogram(name, param.clone().cpu().data.numpy(), epoch)

# Also we can write our hyperparameters either
writer.add_hparams(
    {"lr": lr, "bsize": batch_size, "shuffle": shuffle,},
    {"accuracy": total_correct / len(testset), "loss": loss},
)
writer.close()
print("Finished Training")



100%|██████████| 79/79 [00:03<00:00, 25.18it/s]Finished Training



### Embedding (dimension reduction)

등의 기능들도 존재